In [1]:
from efficientnet_pytorch import EfficientNet

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
from fastai.script import *
from fastai.vision import *
from fastai.callbacks import *
from fastai.distributed import *
from fastprogress import fastprogress
from torchvision.models import *

In [4]:
model = EfficientNet.from_name('efficientnet-b5')

In [5]:
model_name = 'efficientnet-b5'
image_size = EfficientNet.get_image_size(model_name)

In [6]:
image_size

456

In [7]:
model = EfficientNet.from_pretrained(model_name)


Loaded pretrained weights for efficientnet-b5


In [8]:
model

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 48, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        48, 48, kernel_size=(3, 3), stride=[1, 1], groups=48, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        48, 12, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        12, 48, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        48, 24, kernel_siz

In [9]:
np.random.seed(13)


In [25]:
import os
test_file_location = os.getcwd()+'/test-snakes-cropped'

In [26]:
src = (ImageList.from_folder(path='fastai-data').split_by_rand_pct(0.2).label_from_folder())

In [27]:
src.add_test_folder(cwd)

LabelLists;

Train: LabelList (65932 items)
x: ImageList
Image (3, 110, 268),Image (3, 189, 175),Image (3, 232, 405),Image (3, 622, 203),Image (3, 174, 191)
y: CategoryList
class-857,class-857,class-857,class-857,class-857
Path: fastai-data;

Valid: LabelList (16483 items)
x: ImageList
Image (3, 221, 283),Image (3, 192, 320),Image (3, 154, 412),Image (3, 268, 325),Image (3, 115, 79)
y: CategoryList
class-597,class-707,class-204,class-784,class-857
Path: fastai-data;

Test: LabelList (17686 items)
x: ImageList
Image (3, 202, 166),Image (3, 211, 362),Image (3, 367, 563),Image (3, 902, 678),Image (3, 1204, 897)
y: EmptyLabelList
,,,,
Path: fastai-data

In [28]:
tfms=([rotate(degrees=(-90,90), p=0.8)],[])

In [29]:
bs=8

In [30]:
data = (src.transform(tfms, size=image_size, resize_method=ResizeMethod.SQUISH)
        .databunch(bs=bs).normalize(imagenet_stats))

In [ ]:
data.show_batch(3,figsize=(9,9))

In [31]:
list(children(model))[-2:]


[BatchNorm2d(2048, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True),
 Linear(in_features=2048, out_features=1000, bias=True)]

In [32]:
model

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 48, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        48, 48, kernel_size=(3, 3), stride=[1, 1], groups=48, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        48, 12, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        12, 48, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        48, 24, kernel_siz

In [33]:
model.add_module('_fc',nn.Linear(2048, data.c))

In [34]:
loss_func =LabelSmoothingCrossEntropy()


In [35]:
RMSprop = partial(torch.optim.RMSprop)


In [36]:
from torch.optim import Optimizer

In [39]:
learn = Learner(data, model, loss_func=loss_func, opt_func=RMSprop, metrics=[accuracy,FBeta(beta=1,average='macro')])


learn

learn.model

learn.split([[learn.model._conv_stem, learn.model._bn0, learn.model._blocks[:19]],
             [learn.model._blocks[19:],learn.model._conv_head], 
             [learn.model._bn1,learn.model._fc]])

learn.summary()

learn.load('b5-seed-13-round-3')

Learner(data=ImageDataBunch;

Train: LabelList (65932 items)
x: ImageList
Image (3, 456, 456),Image (3, 456, 456),Image (3, 456, 456),Image (3, 456, 456),Image (3, 456, 456)
y: CategoryList
class-857,class-857,class-857,class-857,class-857
Path: fastai-data;

Valid: LabelList (16483 items)
x: ImageList
Image (3, 456, 456),Image (3, 456, 456),Image (3, 456, 456),Image (3, 456, 456),Image (3, 456, 456)
y: CategoryList
class-597,class-707,class-204,class-784,class-857
Path: fastai-data;

Test: LabelList (17686 items)
x: ImageList
Image (3, 456, 456),Image (3, 456, 456),Image (3, 456, 456),Image (3, 456, 456),Image (3, 456, 456)
y: EmptyLabelList
,,,,
Path: fastai-data, model=EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 48, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0

In [40]:
data.c

45

In [ ]:
preds,_ = learn.TTA(ds_type=DatasetType.Test)

In [ ]:
probs = np.exp(preds)/np.exp(preds).sum(1)[:,None]

In [ ]:
probs_np = probs.numpy()

In [ ]:
df_test = pd.read_csv('test_file.csv',low_memory=False)

In [ ]:
df_test.head()

In [ ]:
data.classes[np.argmax(probs[0])]

In [ ]:
df_classes = pd.read_csv('class.csv',low_memory=False)

In [ ]:
df_classes.head()

In [ ]:
data_dict = df_classes.set_index('class_idx')['original_class'].to_dict()

In [ ]:
data_dict

In [ ]:
data.test_ds.items

In [ ]:
print(os.path.basename(data.test_ds.items[0]))

In [ ]:
df_test.head()

In [ ]:
probs_np = probs.numpy()

In [ ]:
df_testfile_mapping = pd.DataFrame(columns=['filename','map'])

In [ ]:
df_testfile_mapping['filename']=df_test['filename']

In [ ]:
for i,row in df_testfile_mapping.iterrows():
    row['map']=i

In [ ]:
df_testfile_mapping.head()

In [ ]:
data_dict_filename = df_testfile_mapping.set_index('filename')['map'].to_dict()

In [ ]:
i = 0
for test_file in data.test_ds.items:
    filename = (os.path.basename(test_file))
    map_val = int(data_dict_filename[filename])    
    for c in range(0,45):
        df_test.loc[map_val,data_dict[int(data.classes[c].split("-")[1])]]=probs_np[i][c]
    i += 1
    print(i)

In [ ]:
df_test.to_csv('submission-b2.csv',index=False)

In [ ]:
for i,row in df_test.iterrows():
    sum_temp=row[1:46].sum()
    low_limit = 1-1e-6
    high_limit = 1+1e-6
    print(sum_temp)
    assert sum_temp>= low_limit and sum_temp <= high_limit

In [ ]:
header=list(range(0,45))

In [ ]:
df_to_save =pd.DataFrame(probs_np)
df_to_save.to_csv('test-b2.csv',header=header,index=False)

In [ ]:
df_final = pd.read_csv('submission-b2.csv',low_memory=False)

In [ ]:
df_final.head()

In [ ]:
for i,row in df_final.iterrows():
    sum_temp=row[1:46].sum()
    low_limit = 1-1e-6
    high_limit = 1+1e-6
    if not (sum_temp>= low_limit and sum_temp <= high_limit):
        print(i)
        print(sum_temp)

In [ ]:
for i,row in df_final.iterrows():
    probs = np.asarray(row[1:46])
    print(i)
    index_val = np.argmax(probs)
    for c in range(1,46):
        df_final.iloc[i,c]=0.44/44 #handles precision error , we set 0.01 to false classes, 0.56 to predicted class
    df_final.iloc[i,index_val+1]=0.56

In [ ]:
for i,row in df_final.iterrows():
    sum_temp=row[1:46].sum()
    low_limit = 1-1e-8
    high_limit = 1+1e-8
    assert sum_temp>= low_limit and sum_temp <= high_limit

In [ ]:
df_final.to_csv('submission-b5-seed-13.csv',index=False)